In [371]:
# Libraries
import numpy as np
rng =np.random.default_rng()
np.set_printoptions(suppress=True, precision=4)
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import OneHotEncoder
from scipy.special import logsumexp
from bokeh.plotting import figure, output_notebook, show
output_notebook()
from tqdm import tqdm

Loading BokehJS ...

## Multiclass Regression

### Simulate data

We will simulate some tiny image data, where each image has is an array of five 0-255 values and there
are 4 different images:
x0x0x
00xxx
xxx00
x000x

In [372]:
samples=[60,40,80,120]
variability=40

x0=[30,200,30,200,30]
x1=[200,200,30,30,30]
x2=[30,30,30,200,200]
x3=[30,200,200,200,30]

data = np.concatenate([rng.normal(loc=x,scale=variability,size=(n,5)).astype(int) for n,x in zip(samples,[x0,x1,x2,x3])])

# cut off entries bigger than 255

labels = np.array([0]*samples[0]+[1]*samples[1]+[2]*samples[2]+[3]*samples[3])

# rearrange the rows
rows = list(range(sum(samples)))
shuffled = rng.shuffle(rows)
data = data[rows,:]
labels=labels[rows]


In [373]:
labels

array([3, 1, 2, 0, 2, 3, 2, 3, 3, 3, 3, 0, 1, 2, 3, 3, 2, 2, 1, 3, 3, 3,
       2, 2, 2, 2, 3, 1, 3, 2, 3, 0, 0, 0, 2, 3, 1, 3, 2, 0, 3, 2, 3, 0,
       3, 3, 2, 2, 0, 2, 0, 3, 2, 3, 1, 0, 1, 0, 1, 0, 3, 1, 2, 2, 3, 2,
       2, 3, 1, 2, 3, 1, 3, 2, 3, 0, 3, 3, 3, 1, 2, 0, 2, 3, 3, 3, 0, 1,
       2, 3, 2, 3, 2, 3, 3, 1, 3, 3, 0, 2, 3, 3, 0, 2, 0, 1, 3, 1, 0, 3,
       1, 3, 3, 0, 2, 0, 1, 1, 2, 0, 2, 2, 2, 2, 3, 3, 3, 2, 2, 1, 3, 3,
       2, 0, 2, 0, 3, 0, 3, 2, 0, 1, 0, 3, 3, 3, 2, 2, 3, 0, 3, 3, 0, 1,
       3, 2, 0, 1, 2, 2, 1, 2, 0, 3, 2, 3, 2, 2, 3, 2, 3, 3, 0, 2, 1, 2,
       2, 3, 3, 3, 0, 1, 3, 3, 3, 0, 0, 2, 3, 0, 3, 0, 3, 0, 3, 3, 2, 2,
       0, 0, 3, 0, 0, 0, 3, 2, 0, 2, 3, 3, 3, 3, 3, 1, 3, 2, 3, 1, 2, 0,
       1, 0, 0, 1, 2, 3, 3, 3, 2, 0, 3, 3, 2, 2, 1, 2, 0, 0, 0, 3, 3, 3,
       0, 1, 1, 2, 2, 3, 2, 0, 3, 1, 3, 3, 3, 0, 3, 1, 3, 3, 3, 3, 3, 2,
       3, 3, 3, 1, 2, 0, 3, 3, 2, 3, 0, 3, 1, 1, 2, 3, 2, 3, 0, 3, 3, 3,
       1, 0, 1, 2, 0, 3, 2, 2, 3, 0, 2, 3, 2, 3])

In [374]:
### Test the sklearn routines

L=LogisticRegression(solver='liblinear')

#### add the constant feature
data = (data-data.mean(axis=0))/data.std(axis=0)
L.fit(data,labels)
print("The coefficient matrix")
print(L.coef_)
print("The intercepts")
print(L.intercept_)


The coefficient matrix
[[-0.7448  1.4407 -3.0763  1.2578 -1.5464]
 [ 1.6059  0.2207 -0.3796 -1.8167 -0.3021]
 [-0.2022 -2.2679 -0.5316  0.2552  2.3482]
 [-0.3142  0.7471  4.2188  0.5531 -0.6291]]
The intercepts
[-2.7463 -3.6326 -2.4263 -1.3264]


In [375]:
print("Accuracy on the test data is {} percent".format(100*L.score(data,labels)))

Accuracy on the test data is 99.0 percent


In [422]:
def sigma(x,m):
    """computes the softmax function on the rows of the data matrix x for the weights m"""
    y = x @ m 
    j = np.exp(y)
    p = j/j.sum(axis=1,keepdims=True)
    return p
# Use sklearn to create a "one-hot" encoding of the labels

E=OneHotEncoder()
Y=E.fit_transform(labels.reshape(-1,1)).toarray()

def descent(x,y, max_iter=10000,nu=.000001,M=None):
    """does gradient ascent to maximum likelihood for data=x and labels (one-hot)=y"""
    features = x.shape[1]
    classes = y.shape[1]
    grads=[]
    if M is None:
        M = rng.normal(loc=0,scale=1,size=(features,classes))
    for i in tqdm(range(max_iter)):
        P=sigma(x,M)
        grad = data.transpose() @ (Y-P)
        M = M+nu*grad
        grads.append(np.max(np.abs(grad)))

    return M,grads

In [423]:
M,g=descent(data,Y,max_iter=10000,nu=1e-5)
print(M)

100%|██████████| 10000/10000 [00:00<00:00, 34120.70it/s]

[[ 0.312   2.2551  0.0543 -0.1579]
 [-0.0391 -1.282  -2.323  -0.3414]
 [-1.993  -0.7251 -0.9839  2.2387]
 [ 1.1905  0.005   0.8452  0.7962]
 [-1.095  -0.8986  0.8361 -1.2064]]


In [424]:
### the predicted label is the one with maximum probability
predictions=np.argmax(sigma(data,M),axis=1)

### accuracy compares predicted to true
correct=((predictions==labels).sum())/predictions.shape[0]

print("Accuracy here is {} percent".format(100* correct))

Accuracy here is 98.33333333333333 percent


In [425]:
f=figure(title='Size of largest coefficient in gradient vs. number of iterations')
f.line(x=list(range(len(g))),y=g)
show(f)

In [426]:
np.min(g)

8.84359279700982

In [443]:
M,g=descent(data,Y,max_iter=100000,nu=1e-5)

100%|██████████| 100000/100000 [00:02<00:00, 34717.29it/s]


In [444]:
f=figure(title='Size of largest coefficient in gradient vs. number of iterations')
f.line(x=list(range(len(g))),y=g)
show(f)

In [445]:
g[-1]

1.0148803989102506

In [455]:
M,g=descent(data,Y,max_iter=10000,nu=1e-5)
bigg=[]
for i in range(50):
    M,g=descent(data,Y,max_iter=10000,nu=(1e-5)/2,M=M)
    bigg = bigg + g
    print(g[-1])
    

 

100%|██████████| 10000/10000 [00:00<00:00, 34738.17it/s]


5.239145689147484


100%|██████████| 10000/10000 [00:00<00:00, 34483.44it/s]


3.9878568557014606


100%|██████████| 10000/10000 [00:00<00:00, 35510.37it/s]


...

0.49032503962727825


100%|██████████| 10000/10000 [00:00<00:00, 34968.82it/s]


0.48297739559155944


100%|██████████| 10000/10000 [00:00<00:00, 34965.30it/s]

0.4758872054455315


In [456]:
M

array([[-1.3558,  1.4395, -0.5234, -0.7165],
       [ 2.073 ,  0.6709, -2.088 ,  2.2809],
       [-4.5633,  0.0461, -0.9698,  5.4576],
       [ 1.6446, -1.8624,  0.5536,  0.871 ],
       [-1.89  ,  0.1597,  3.23  , -1.538 ]])